In [2]:
import sys
import gc
sys.path.append('/usr/local/lib/python3.6/site-packages')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score, mean_squared_error, make_scorer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder


import lightgbm as lgb
#from hyperopt import hp, tpe
#from hyperopt.fmin import fmin

gc.collect()

7

In [2]:
def prepare_full():
    train = pd.read_csv('../datasets/train.csv', encoding='cp1251')
    test = pd.read_csv('datasets/test.csv', encoding='cp1251')
    
    train['is_train'] = 1
    test['is_train'] = 0
    
    full = pd.concat([train,test])
    
    del train, test
    gc.collect()
    
    le = LabelEncoder()
    full['bulk_id_int'] = le.fit_transform(full['bulk_id'])
    full['date1'] = pd.to_datetime(full['date1'], format='%Y-%m-%d')
    GLOBAL_MINDATE = full['date1'].min()
    full['Date_int'] = ((full['date1'] - GLOBAL_MINDATE)/np.timedelta64(1, 'D')).astype('int32')

    full['Автомойка'] = (full['Автомойка']=='да').astype('int')
    full['Входные группы'] = (full['Входные группы']=='да').astype('int')
    full['Двор без машин'] = (full['Двор без машин']=='да').astype('int')
    full['Класс объекта'] = full['Класс объекта'].map({'эконом':1, 'комфорт':3, 'стандарт':2})
    full['Кладовые'] = (full['Кладовые']=='да').astype('int')
    full['Колясочные'] = (full['Колясочные']=='да').astype('int')
    full['Огорожена территория'] = (full['Огорожена территория']=='да').astype('int')
    full['Подземная парковка'] = (full['Подземная парковка']=='да').astype('int')
    full['Система мусоротведения'] = le.fit_transform(full['Система мусоротведения'])
    full['Спортивная площадка'] = (full['Спортивная площадка']=='да').astype('int')
    
    #введем уникальные id
    full['bulk_spalen_id'] = full['bulk_id_int'].astype('str')+'_'+full['spalen'].astype('str')
    full['bulk_spalen_id'] = le.fit_transform(full['bulk_spalen_id'])
    
    full = full.sort_values('month_cnt')
    
    # подсчитаем псевдо start_square (без учета возвращенных)
    full['calc_start_square'] = full.groupby(['bulk_spalen_id'])['start_square'].shift(1) - full.groupby(['bulk_spalen_id'])['value'].shift(1)
    full['calc_last_value'] = full.groupby(['bulk_spalen_id'])['value'].shift(1)
    
    full['date2'] = full.date1+ pd.offsets.MonthEnd(1)
    
    full['price_by_square'] = full['price']/full['mean_sq']
    
    return full
    

In [3]:
def prepare_flat():
    le = LabelEncoder()
    flat = pd.read_csv('datasets/flat.csv', encoding='cp1251')
    flat = flat.rename(columns = {'id_bulk':'bulk_id'})
    flat['id_flatwork_int'] = np.array(flat.index).astype('int')
    
    dict_bulk_spalen = full.loc[:, ('bulk_id','spalen','bulk_spalen_id')] \
                       .drop_duplicates() 
    dict_flat = flat[['id_flatwork_int','id_flatwork','bulk_id','spalen']].copy()

    dict_flat = dict_flat.merge(dict_bulk_spalen, how = 'left')
    
    
    flat['Автомойка'] = (flat['Автомойка']=='да').astype('int')
    flat['Входные группы'] = (flat['Входные группы']=='да').astype('int')
    flat['Двор без машин'] = (flat['Двор без машин']=='да').astype('int')
    flat['Класс объекта'] = flat['Класс объекта'].map({'эконом':1, 'комфорт':3, 'стандарт':2})
    flat['Кладовые'] = (flat['Кладовые']=='да').astype('int')
    flat['Колясочные'] = (flat['Колясочные']=='да').astype('int')
    flat['Огорожена территория'] = (flat['Огорожена территория']=='да').astype('int')
    flat['Подземная парковка'] = (flat['Подземная парковка']=='да').astype('int')
    flat['Система мусоротведения'] = le.fit_transform(flat['Система мусоротведения'])
    flat['Спортивная площадка'] = (flat['Спортивная площадка']=='да').astype('int')
    flat['otdelka'] = le.fit_transform(flat['otdelka'].fillna('nan'))
    flat['vid'] = flat['vid'].map({'эконом':1, 'средний':2, 'хороший':3}).fillna(0)
    flat['plan_size'] = flat['plan_size'].map({'S':1, 'M':2, 'L':3, '-1':0})
    flat['plan0'] = le.fit_transform(flat['plan0'].fillna('nan'))

    flat['date_settle'] = pd.to_datetime(flat['date_settle'], format='%Y-%m-%d')
    flat['date_salestart'] = pd.to_datetime(flat['date_salestart'], format='%Y-%m-%d')

    flat.loc[~flat['date_settle'].isna(),'dt_settle_salestart'] = ((flat.loc[~flat['date_settle'].isna(),'date_settle'] - flat.loc[~flat['date_settle'].isna(),'date_salestart'])/np.timedelta64(1, 'D')).astype('int32')

    #заполним медианным значением
    dt_settle_salestart_median = int(flat.loc[~flat['date_settle'].isna(),'dt_settle_salestart'].median())
    flat.loc[flat['date_settle'].isna(),'dt_settle_salestart'] = dt_settle_salestart_median  
    
    flat['dt_settle_salestart'] = flat['dt_settle_salestart'].astype('int32')
    
    flat.loc[flat['date_settle'].isna(),'date_settle'] = flat.loc[flat['date_settle'].isna(),'date_salestart'] + np.timedelta64(dt_settle_salestart_median, 'D')
    
    
    flat = flat.merge(dict_flat[['id_flatwork_int','bulk_spalen_id']], how = 'left', on = 'id_flatwork_int')
     
    return flat, dict_bulk_spalen, dict_flat
    

In [4]:
full = prepare_full()

In [5]:
flat, dict_bulk_spalen, dict_flat = prepare_flat()


In [6]:
price = pd.read_csv('datasets/price.csv', encoding='utf-8')
status = pd.read_csv('datasets/status.csv', encoding='cp1251')

price = price.merge(dict_flat, how = 'inner')
status = status.merge(dict_flat, how = 'inner')


dict_stat = status[['stat','stat_name']].drop_duplicates()
dict_stat['can_be_sold'] = (~dict_stat.stat_name.isin(['Реализован','Статус после покупки'])).astype('int')
dict_stat['sold'] = (dict_stat.stat_name.isin(['Реализован','Статус после покупки'])).astype('int')
dict_stat['realize'] = (dict_stat.stat_name.isin(['Реализован'])).astype('int')
dict_stat['stat_new'] = dict_stat['stat_name'].map({'Не реализуется':0,
                                                    'В реализации (не на сайте)':1,
                                                    'В реализации':2,
                                                    'Онлайн бронирование':3,
                                                    'Зарезервирован под клиента':3,
                                                    'Платное бронирование':4,
                                                    'Реализован':5,
                                                    'Статус после покупки':6
                                                    })

status = status.merge(dict_stat[['stat','stat_new']], how = 'inner')

status = status.sort_values(['datefrom','dateto'])
status['last_stat_new'] = status.groupby(['id_flatwork_int'])['stat_new'].shift(1)#.fillna('stat_new')
status.loc[status['last_stat_new'].isna(),'last_stat_new'] = -1

gc.collect()

128

In [7]:
def my_drop_levels(dataset, sep = '_', brief = ''):
    if dataset.columns.nlevels>1:
        new_columns = np.array([], dtype = 'str')
        for col_i in range(dataset.shape[1]):
            col_name = brief
            for level in range(dataset.columns.nlevels):
                tmp_col_name = dataset.columns.levels[level][dataset.columns.labels[level][col_i]]#.astype('str')
                tmp_col_name = str(tmp_col_name) 
                if (level>0) & (tmp_col_name!=''):
                    col_name = col_name+'_'
                col_name = col_name+tmp_col_name
            new_columns = np.append(new_columns,col_name)
            #print(col_name)
        for level in range(dataset.columns.nlevels-1):
            dataset.columns.droplevel(0)
        dataset.columns = new_columns      
    return dataset

# Формируем поквартирную обучающую выборку

In [8]:
%%time

fixed_days = np.sort(full.date1.dt.strftime('%Y-%m-%d').unique())
fixed_dates = np.sort(full.date1.dt.strftime('%Y-%m-%d').unique() + ' 21:00:00')
fixed_dates_last = np.sort(full.date2.dt.strftime('%Y-%m-%d').unique() + ' 21:00:00')


for i in range(len(fixed_days)):
    gc.collect()
    
    fixed_day = fixed_days[i]
    fixed_date = fixed_dates[i] 
    fixed_date_last = fixed_dates_last[i]


    #найдем квартиры, доступные к продаже на эту дату
    status_on_date = status[(status.datefrom<=fixed_date) & (status.dateto>fixed_date)].copy()
    

    #срок жизни статуса
    status_on_date['datefrom'] = pd.to_datetime(status_on_date['datefrom'], format='%Y-%m-%d %H:%M:%S')
    status_on_date['datenow'] = pd.to_datetime(fixed_date, format='%Y-%m-%d %H:%M:%S')
    #status_on_date['dateto'] = pd.to_datetime(status_on_date['dateto'], format='%Y-%m-%d %H:%M:%S')
    
    status_on_date['status_days'] = ((status_on_date['datenow'] - status_on_date['datefrom'])/np.timedelta64(1, 'D')).astype('int32')
    
    #подсчитаем все статусы, которые были у этой квартиры к указанной дате
    statuses_to_date = status[(status.datefrom<=fixed_date)] \
                    .groupby(['id_flatwork_int','stat_new']) \
                    .size() \
                    .reset_index(name = 'cnt_stat_new')
    statuses_to_date = statuses_to_date.pivot(index = 'id_flatwork_int', columns='stat_new').fillna(0)
    statuses_to_date = my_drop_levels(statuses_to_date, sep = '_') 
    statuses_to_date = statuses_to_date.reset_index()
    
    #удалим задвоения
    tmp = status_on_date.groupby('id_flatwork_int').size().reset_index(name = 'cnt')
    flats_to_delete =  tmp.loc[tmp['cnt']>1,'id_flatwork_int']
    status_on_date = status_on_date[~status_on_date.id_flatwork_int.isin(flats_to_delete)] 
    
    #квартиры, которые могут быть проданы в этом периоде
    stats_can_be_sold = dict_stat[dict_stat['can_be_sold']==1].stat
    flats_can_be_sold = status_on_date[status_on_date.stat.isin(stats_can_be_sold)].id_flatwork_int
    
    #формируем простейшую поквартирную обучающую выборку
    tmp_flat_train = flat[(~flat.bulk_spalen_id.isna()) & 
                          (flat.id_flatwork_int.isin(flats_can_be_sold)) #| flat.id_flatwork_int.isin(flats_sold_by_period)
                         ].copy()
    
    #за ближайший месяц
    # таргет - поле sale за период
    flats_sold_by_period = flat[(flat['sale'] >= fixed_date) & (flat['sale'] <= fixed_date_last)] \
                                    .id_flatwork_int \
                                    .unique()                  
    tmp_flat_train['realized_1'] = (tmp_flat_train.id_flatwork_int.isin(flats_sold_by_period)).astype('int')
    tmp_flat_train['value_1'] = tmp_flat_train['square']*tmp_flat_train['realized_1']
    
    if (i+1)<len(fixed_days):
        # таргет - наличие статуса реализован за период
        flats_sold_by_period = flat[(flat['sale'] >= fixed_dates[i+1]) & (flat['sale'] <= fixed_dates_last[i+1])] \
                                        .id_flatwork_int \
                                        .unique()                  
        tmp_flat_train['realized_2'] = (tmp_flat_train.id_flatwork_int.isin(flats_sold_by_period)).astype('int')
        tmp_flat_train['value_2'] = tmp_flat_train['square']*tmp_flat_train['realized_2']
    
    if (i+2)<len(fixed_days):
        # таргет - наличие статуса реализован за период
        flats_sold_by_period = flat[(flat['sale'] >= fixed_dates[i+2]) & (flat['sale'] <= fixed_dates_last[i+2])] \
                                        .id_flatwork_int \
                                        .unique()                  
        tmp_flat_train['realized_3'] = (tmp_flat_train.id_flatwork_int.isin(flats_sold_by_period)).astype('int')
        tmp_flat_train['value_3'] = tmp_flat_train['square']*tmp_flat_train['realized_3']
    

    tmp_flat_train = tmp_flat_train.merge(status_on_date[['id_flatwork_int','stat_new','last_stat_new','status_days']], how = 'inner', on = 'id_flatwork_int')
    tmp_flat_train = tmp_flat_train.merge(statuses_to_date, how = 'left', on = 'id_flatwork_int')

    
    tmp_flat_train['date1']=fixed_day 
    tmp_flat_train['month_cnt']=i
    tmp_flat_train['dt_to_settle'] = ((pd.to_datetime(tmp_flat_train['date1'], format='%Y-%m-%d')  - tmp_flat_train['date_settle'])/np.timedelta64(1, 'D')).astype('int32')
    tmp_flat_train['dt_to_salestart'] = ((pd.to_datetime(tmp_flat_train['date1'], format='%Y-%m-%d') - tmp_flat_train['date_salestart'])/np.timedelta64(1, 'D')).astype('int32')
   
    #тут стоит поиграться со статусами, историей, ценой
    price_on_date = price[(price.datefrom<=fixed_date) & (price.dateto>fixed_date)]
    tmp_flat_train = tmp_flat_train.merge(price_on_date[['id_flatwork_int','pricem2']], how = 'inner', on = 'id_flatwork_int')
    
    #начнем со статусов
    
    
    if i==0:
        flat_train = tmp_flat_train.fillna(0)
    else:
        flat_train = flat_train.append(tmp_flat_train.fillna(0))
    
    #if i==19:
    #    break
    #break
    
flat_train = flat_train.fillna(0)          

CPU times: user 34.8 s, sys: 3.11 s, total: 37.9 s
Wall time: 37.8 s


In [9]:
def mem_economy(dataset):
    features = np.array(dataset.dtypes[dataset.dtypes=='float64'].index)
    for f in features:
        dataset[f] = dataset[f].astype('float32')
    features = np.array(dataset.dtypes[dataset.dtypes=='int64'].index)
    for f in features:
        dataset[f] = dataset[f].astype('int32')
    
    return dataset


In [10]:
full = mem_economy(full)
gc.collect()
flat = mem_economy(flat)
gc.collect()
flat_train = mem_economy(flat_train)
gc.collect()


20

In [11]:
def my_submit(model, 
                        dataset, 
                        right_dataset, 
                        right_date, 
                        cv_dates,
                        last_date,
                        n_month,
                        study_columns, 
                        value_column, 
                        group_columns, 
                        random_state=442, 
                        importance_flag = False):
    
    #весь обучающий датасет
    train_agg = dataset.copy().reset_index(drop = True)
    
    ind = 0
    _mse = np.array([],dtype = 'float')
    _grp_mse = np.array([],dtype = 'float')
    gc.collect()
    print('==========================')
    
    #основная кросс-валидация
    d = cv_dates[len(cv_dates)-1]
     
    #Расчитаем для submit-а

    #обучающая сдвигается на 1 месяц вперед
    dt = fixed_days[d+1]
    if d+1-n_month<0:
        dt_start = fixed_days[0]
    else:
        dt_start = fixed_days[d+1-n_month]
            
    print('study dataset fot test: date = ',dt,' dt_start = ',dt_start)
        
    tmp_train  = train_agg.loc[(train_agg.date1<dt) & (train_agg.date1>=dt_start),:] 
    #а тестовая - на последнюю известную дату
    tmp_test  = train_agg.loc[train_agg.date1==last_date,:]   
    tmp_right = right_dataset.loc[right_dataset.date1==right_date,:].copy()
        
    #учиться будем только на study_columns не на всех переменных     
    X_train = tmp_train.loc[:,study_columns]
    X_test  = tmp_test.loc[:,study_columns]
        
    y_train = tmp_train[value_column]
        
    del tmp_train
    gc.collect()
        
    #обучим модель
    model.fit(X_train,y_train)
        
    y_test_pred = model.predict(X_test)
    y_test_pred[y_test_pred<0] = 0
        
    R_test = X_test.copy()
    R_test['predict'] = y_test_pred 
             
    R_test = R_test.groupby(group_columns) \
                             .agg({'predict':'sum'}) \
                             .reset_index()
    tmp_right = tmp_right.merge(R_test, on = group_columns, how = 'left')
    submission = tmp_right[['id','predict']].rename(columns = {'predict':'value'}).fillna(0)
    

    return submission#, _mse, _grp_mse#, importance, model, full_df_for_calc_cv

In [12]:
%%time

gc.collect()
#определим цену продажи
column_filter = ['id_sec','id_gk','id_flatwork','date_settle', 
                 'date_salestart','sale','bulk_id',
                 'date1','realized_1', 'realized_2', 'realized_3',
                 'value_1','value_2', 'value_3']

                
column_study = np.setdiff1d(np.asarray(flat_train.columns), column_filter)




#n_month = 15
last_date = '2018-02-01'

for i in range(5):
    n_month = 15
    lgb_model = lgb.LGBMRegressor(n_estimators = 200, random_state = 42+i, predict_leaf_index = True)
    
    submission_1 = my_submit(
                            model = lgb_model, 
                            dataset = flat_train,
                            right_dataset = full[['id','is_train','bulk_spalen_id','value','date1']],
                            right_date = fixed_days[31], 
                            cv_dates = [30], 
                            last_date = last_date,
                            n_month = n_month,
                            study_columns = column_study, 
                            value_column = 'value_1', 
                            group_columns = 'bulk_spalen_id',
                            random_state=442, 
                            importance_flag = True)

    submission_2 = my_submit(
                            model = lgb_model, 
                            dataset = flat_train,
                            right_dataset = full[['id','is_train','bulk_spalen_id','value','date1']],
                            right_date = fixed_days[32], 
                            cv_dates = [29],
                            last_date = last_date,
                            n_month = n_month,
                            study_columns = column_study, 
                            value_column = 'value_2', 
                            group_columns = 'bulk_spalen_id',
                            random_state=442, 
                            importance_flag = True)

    submission_3 = my_submit(
                            model = lgb_model, 
                            dataset = flat_train,
                            right_dataset = full[['id','is_train','bulk_spalen_id','value','date1']],
                            right_date = fixed_days[33],
                            cv_dates = [28],
                            last_date = last_date,
                            n_month = n_month,
                            study_columns = column_study, 
                            value_column = 'value_3', 
                            group_columns = 'bulk_spalen_id',
                            random_state=442, 
                            importance_flag = True)


    submission = pd.concat([submission_1,submission_2,submission_3]).fillna(0).sort_values('id')
    
    if i==0:
        v = submission['value']
    else: 
        v = v + submission['value']
        
submission['value'] = v/(i+1)        

filename = f'results/afterparty/x_5_nmonth_15.csv'
#submission.to_csv(filename, index = False)


study dataset fot test: date =  2018-02-01  dt_start =  2016-11-01
study dataset fot test: date =  2018-01-01  dt_start =  2016-10-01
study dataset fot test: date =  2017-12-01  dt_start =  2016-09-01
study dataset fot test: date =  2018-02-01  dt_start =  2016-11-01
study dataset fot test: date =  2018-01-01  dt_start =  2016-10-01
study dataset fot test: date =  2017-12-01  dt_start =  2016-09-01
study dataset fot test: date =  2018-02-01  dt_start =  2016-11-01
study dataset fot test: date =  2018-01-01  dt_start =  2016-10-01
study dataset fot test: date =  2017-12-01  dt_start =  2016-09-01
study dataset fot test: date =  2018-02-01  dt_start =  2016-11-01
study dataset fot test: date =  2018-01-01  dt_start =  2016-10-01
study dataset fot test: date =  2017-12-01  dt_start =  2016-09-01
study dataset fot test: date =  2018-02-01  dt_start =  2016-11-01
study dataset fot test: date =  2018-01-01  dt_start =  2016-10-01
study dataset fot test: date =  2017-12-01  dt_start =  2016-0

# Само обучение на квартирах

# Модель на train.csv и test.csv

In [13]:
def my_one_model_cv(model, dataset, study_columns, random_state=442, importance_flag = False):
    
    train_agg = dataset[dataset.is_train==1].copy()
    test_agg = dataset[dataset.is_train==0].copy()
    
    ind = 0
    _mse = np.array([],dtype = 'float')
    #заполним нулями предикт теста
    y_test_pred = np.zeros(test_agg.shape[0],dtype = 'float')

    #основная кросс-валидация
    for train_index, valid_index in KFold(n_splits=5, random_state=random_state, shuffle = True).split(train_agg):   

        tmp_train  = train_agg.loc[train_index,:]   
        tmp_valid  = train_agg.loc[valid_index,:]
        tmp_test   = test_agg.copy()

        #учиться будем только на study_columns не на всех переменных     
        X_train = tmp_train.loc[:,study_columns]
        X_valid = tmp_valid.loc[:,study_columns]
        X_test  = tmp_test.loc[:,study_columns]

        y_train = tmp_train['value']
        y_valid = tmp_valid['value']
        
        #обучим модель
        model.fit(X_train,y_train)
        

        
        y_valid_pred = model.predict(X_valid)
        y_test_pred = y_test_pred+model.predict(X_test)
        
        
        y_valid_pred[y_valid_pred<0] = 0
        
        if ind ==0:
            stacking_df = pd.DataFrame(dict({'bulk_id_int':tmp_valid.bulk_id_int,'predict':y_valid_pred, 'fact':y_valid}))
        else:
            tmp_stacking_df = pd.DataFrame(dict({'bulk_id_int':tmp_valid.bulk_id_int,'predict':y_valid_pred, 'fact':y_valid}))
            stacking_df = stacking_df.append(tmp_stacking_df).sort_values('bulk_id_int')
        
        
        _mse = np.append(_mse,mean_squared_error(y_valid,y_valid_pred))

        ind = ind + 1

        #break
    
    
    importance = pd.DataFrame(dict({'feature':'none', 'delta_mse':0}), index = ['none'])
    
    mse_now = mean_squared_error(y_valid,y_valid_pred)
    NUMBER_SHUFFLE = 5
    if importance_flag:
        for feature in study_columns:

            tmp_mse = 0
            for i in range(NUMBER_SHUFFLE):
                _X_valid = X_valid.copy()
                a = np.asarray(X_valid[feature].copy())
                np.random.shuffle(a)
                _X_valid[feature] = a
                y_valid_pred = model.predict(_X_valid)
                tmp_mse = tmp_mse+mean_squared_error(y_valid, y_valid_pred)/NUMBER_SHUFFLE
            tmp_importance = pd.DataFrame(dict({'feature':feature, 'delta_mse':(tmp_mse-mse_now)}), index = [feature])    
            importance = importance.append(tmp_importance) 
    
    
    #усредняем по фолдам предсказание теста
    y_test_pred = y_test_pred/ind
    
    y_test_pred[y_test_pred<0] = 0
    
    submission = pd.DataFrame(dict({'id':test_agg.id,'value':y_test_pred}))
    
    
    
    return submission, _mse, stacking_df, importance

In [14]:
full['price_by_square'] = full['price']/full['mean_sq']

In [15]:
#column_filter = ['value','start_square','plan_s','plan_m','plan_l','vid_0','vid_1','vid_2',
#                 'date1','bulk_id','is_train','id','date2']
                 #,'calc_last_value', 'calc_start_square']
#column_study = np.setdiff1d(np.asarray(full.columns), column_filter)

column_study = np.array(['Cтавка по ипотеке', 'Date_int', 'bulk_id_int', 'mean_fl',
       'mean_sq', 'month', 'month_cnt', 'price', 'price_by_square',
       'spalen', 'Автомойка', 'Вентлияция', 'Видеонаблюдение',
       'Вклады до 1 года', 'Вклады от 1 года до 3 лет',
       'Вклады свыше 3 лет', 'Входные группы', 'Двор без машин',
       'Детский сад', 'До Кремля', 'До Садового(км)', 'До ТТК(км)',
       'До большой дороги на машине(км)', 'До метро пешком(км)',
       'До парка пешком(км)', 'До парка(км)', 'До промки(км)',
       'До удобной авторазвязки на машине(км)', 'Кладовые',
       'Класс объекта', 'Количество помещений', 'Колясочные',
       'Кондиционирование', 'Курс', 'Лифт', 'Машиномест',
       'Огорожена территория', 'Площадь двора',
       'Площадь зеленой зоны в радиусе 500 м',
       'Площадь земельного участка', 'Площадь пром. зоны в радиусе 500 м',
       'Подземная парковка', 'Поликлиника', 'Система мусоротведения',
       'Спортивная площадка', 'Станций метро от кольца', 'ФОК', 'Школа'])

lgb_model = lgb.LGBMRegressor(n_estimators = 550, random_state = 42)


for i in range(5):
      

    submission_lgb, mse, stacking_df, imp_df = my_one_model_cv(lgb_model, full, column_study, random_state=442, importance_flag = True)

    submission_lgb = submission_lgb.sort_values('id')
    if i==0:
        v = submission_lgb['value']
    else: 
        v = v + submission_lgb['value']
        
submission_lgb['value'] = v/(i+1)   

rmse = np.sqrt(mse)


submission_lgb = submission_lgb.sort_values('id')
filename = f'results/lgb_x_5.csv'
#submission_lgb.to_csv(filename, index = False)

In [16]:
best_lgb = submission_lgb
best_flat = submission


best = best_flat.merge(best_lgb, on = 'id')
best.loc[best['value_x']==0, 'value_x'] = best.loc[best['value_x']==0, 'value_y']
best['value'] = 0.7*best['value_x']+0.3*best['value_y']


filename = f'results/afterparty/blend_auto_x5.csv'
best[['id','value']].to_csv(filename, index = False)




In [28]:
column_study

array(['balcon', 'bulk_spalen_id', 'cnt_stat_new_0', 'cnt_stat_new_1',
       'cnt_stat_new_2', 'cnt_stat_new_3', 'cnt_stat_new_4',
       'cnt_stat_new_5', 'cnt_stat_new_6', 'dt_settle_salestart',
       'dt_to_salestart', 'dt_to_settle', 'floor', 'id_flatwork_int',
       'last_stat_new', 'month_cnt', 'otdelka', 'plan0', 'plan_size',
       'pricem2', 'section', 'spalen', 'square', 'stage_number',
       'stat_new', 'status_days', 'vid', 'Автомойка', 'Вентлияция',
       'Видеонаблюдение', 'Входные группы', 'Двор без машин',
       'Детский сад', 'До Кремля', 'До Садового(км)', 'До ТТК(км)',
       'До большой дороги на машине(км)', 'До метро пешком(км)',
       'До парка пешком(км)', 'До парка(км)', 'До промки(км)',
       'До удобной авторазвязки на машине(км)', 'Кладовые',
       'Класс объекта', 'Количество помещений', 'Колясочные',
       'Кондиционирование', 'Лифт', 'Машиномест', 'Огорожена территория',
       'Площадь двора', 'Площадь зеленой зоны в радиусе 500 м',
       'Пло

In [31]:
flat_train.groupby('date1').size()

date1
2015-07-01     6130
2015-08-01     8256
2015-09-01     8971
2015-10-01     9506
2015-11-01    10183
2015-12-01     9604
2016-01-01     9408
2016-02-01     9197
2016-03-01    10107
2016-04-01    12433
2016-05-01    12541
2016-06-01    12131
2016-07-01    12087
2016-08-01    11942
2016-09-01    12117
2016-10-01    12222
2016-11-01    11469
2016-12-01    38890
2017-01-01    42313
2017-02-01    19884
2017-03-01    21677
2017-04-01    23206
2017-05-01    26751
2017-06-01    26912
2017-07-01    25459
2017-08-01    24996
2017-09-01    27433
2017-10-01    28498
2017-11-01    26913
2017-12-01    27247
2018-01-01    25887
2018-02-01    25863
2018-03-01    25863
2018-04-01    25863
dtype: int64

In [17]:
#best_1 = pd.read_csv('results/send/blend_auto.csv', encoding='cp1251')
#best_2 = pd.read_csv('results/send/blend_auto_x5.csv', encoding='cp1251')

#best = best_1.merge(best_2, on = 'id')
#best['value'] = 0.5*best['value_x']+0.5*best['value_y']

#filename = f'results/blend_blend_x5.csv'
#best[['id','value']].to_csv(filename, index = False)




In [27]:
column_filter = ['id_sec','id_gk','id_flatwork','date_settle', 
                 'date_salestart','sale','bulk_id',
                 'date1','realized_1', 'realized_2', 'realized_3',
                 'value_1','value_2', 'value_3']

                
column_study = np.setdiff1d(np.asarray(flat_train.columns), column_filter)
column_study

array(['balcon', 'bulk_spalen_id', 'cnt_stat_new_0', 'cnt_stat_new_1',
       'cnt_stat_new_2', 'cnt_stat_new_3', 'cnt_stat_new_4',
       'cnt_stat_new_5', 'cnt_stat_new_6', 'dt_settle_salestart',
       'dt_to_salestart', 'dt_to_settle', 'floor', 'id_flatwork_int',
       'last_stat_new', 'month_cnt', 'otdelka', 'plan0', 'plan_size',
       'pricem2', 'section', 'spalen', 'square', 'stage_number',
       'stat_new', 'status_days', 'vid', 'Автомойка', 'Вентлияция',
       'Видеонаблюдение', 'Входные группы', 'Двор без машин',
       'Детский сад', 'До Кремля', 'До Садового(км)', 'До ТТК(км)',
       'До большой дороги на машине(км)', 'До метро пешком(км)',
       'До парка пешком(км)', 'До парка(км)', 'До промки(км)',
       'До удобной авторазвязки на машине(км)', 'Кладовые',
       'Класс объекта', 'Количество помещений', 'Колясочные',
       'Кондиционирование', 'Лифт', 'Машиномест', 'Огорожена территория',
       'Площадь двора', 'Площадь зеленой зоны в радиусе 500 м',
       'Пло

In [18]:
#best_lgb = submission_lgb
#best_flat = submission


#best = best_flat.merge(best_lgb, on = 'id')
#best.loc[best['value_x']==0, 'value_x'] = best.loc[best['value_x']==0, 'value_y']
#best['value'] = 0.7*best['value_x']+0.3*best['value_y']

#idea4 = pd.read_csv('results/send/idea4_mse_15.5509_0.0000.csv', encoding='cp1251')

#best = best[['id','value']].merge(idea4, on = 'id')
#best['value'] = 0.85*best['value_x']+0.15*best['value_y']


#filename = f'results/blend_auto_x5_idea4_085.csv'
#best[['id','value']].to_csv(filename, index = False)



